In [1]:
import pymongo
import pandas as pd
import numpy as np
from sklearn import datasets
import pickle
import time
import pymongo
import numpy as np
import pandas as pd 
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
import random
from pymongo import MongoClient
import pandas as pd
import os
import re
import base64
import numpy as np

from PIL import Image
from io import BytesIO

Using TensorFlow backend.


# Loading Model 

In [2]:
def load_saved_model_from_db(model_name, client, db, dbconnection):
    json_data = {}
    
    #saving model to mongoDB
    # creating connection
    myclient = pymongo.MongoClient(client)
    
    #creating database in mongodb
    mydb = myclient[db]
    
    #creating collection
    mycon = mydb[dbconnection]
    data = mycon.find({'name': model_name})
    
    
    for i in data:
        json_data = i
    #fetching model from db
    pickled_model = json_data[model_name]
    
    return pickle.loads(pickled_model)

In [3]:
details = {
        'inserted_id':"ObjectId('5eb3e0954668994dc15066e2')",
        'model_name':"CNN",
        'created_time':1588884747.028385 }

In [4]:
model  = load_saved_model_from_db(model_name = details['model_name'], client = 'mongodb://localhost:27017/', 
                         db = 'Cat_dog_classifier', dbconnection = 'Model_Accuracy_95.0%')

In [5]:
IMAGE_WIDTH=224
IMAGE_HEIGHT=224
IMAGE_SIZE=[IMAGE_WIDTH, IMAGE_HEIGHT]
IMAGE_CHANNELS=3
batch_size=15

# Test Data 

In [6]:
test_filenames = os.listdir("test1")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

In [7]:
test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "test1/", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    shuffle=False
)

Found 12500 validated image filenames.


In [8]:
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))

In [9]:
test_df['category'] = np.argmax(predict, axis=-1)

In [10]:
test_df

,filename,category
0,11835.jpg,1
1,4737.jpg,1
2,3323.jpg,1
3,4758.jpg,0
4,5678.jpg,0
...,...,...
12495,6422.jpg,1
12496,3040.jpg,0
12497,10816.jpg,0
12498,2124.jpg,1


In [11]:
lables = {('cat', 0), ('dog', 1)}

In [12]:
label_map = dict((v,k) for k,v in lables)
test_df['category'] = test_df['category'].replace(label_map)

In [13]:
test_df

,filename,category
0,11835.jpg,dog
1,4737.jpg,dog
2,3323.jpg,dog
3,4758.jpg,cat
4,5678.jpg,cat
...,...,...
12495,6422.jpg,dog
12496,3040.jpg,cat
12497,10816.jpg,cat
12498,2124.jpg,dog


In [14]:
pro = []
for index, probability in enumerate(predict):
    probability = probability*100
    pro.append(probability)
    
prob = pd.DataFrame(pro)
prob.columns = ['Dog','Cat']
prob.columns

Index(['Dog', 'Cat'], dtype='object')

In [15]:
def save(x):
    if x['Dog'] >= x['Cat']:
        return x['Dog']
    else:
        return x['Cat']
prob['Prediction'] = prob.apply(save, axis=1)

In [16]:
prob

,Dog,Cat,Prediction
0,0.0,1.000000e+02,100.0
1,0.0,1.000000e+02,100.0
2,0.0,1.000000e+02,100.0
3,100.0,0.000000e+00,100.0
4,100.0,0.000000e+00,100.0
...,...,...,...
12495,0.0,1.000000e+02,100.0
12496,100.0,0.000000e+00,100.0
12497,100.0,0.000000e+00,100.0
12498,0.0,1.000000e+02,100.0


In [17]:
df = pd.concat([test_df,prob],axis=1)

In [18]:
df.columns

Index(['filename', 'category', 'Dog', 'Cat', 'Prediction'], dtype='object')

In [19]:
from time import gmtime, strftime
from datetime import datetime

Time = []
Date = [] 




for i in  range(len(df["category"])):
    time = datetime.now().strftime('%H:%M:%S')
    Time.append(time)
    
for i in  range(len(df["category"])):
    date = datetime.now().strftime('%Y-%m-%d')
    Date.append(date)
    
    
Time = pd.DataFrame(Time)
Date = pd.DataFrame(Date)


Time.columns = ["Time"]
Date.columns = ["Date"]

df = pd.concat([df,Time,Date],axis=1)

df

,filename,category,Dog,Cat,Prediction,Time,Date
0,11835.jpg,dog,0.0,1.000000e+02,100.0,16:44:42,2020-05-10
1,4737.jpg,dog,0.0,1.000000e+02,100.0,16:44:42,2020-05-10
2,3323.jpg,dog,0.0,1.000000e+02,100.0,16:44:42,2020-05-10
3,4758.jpg,cat,100.0,0.000000e+00,100.0,16:44:42,2020-05-10
4,5678.jpg,cat,100.0,0.000000e+00,100.0,16:44:42,2020-05-10
...,...,...,...,...,...,...,...
12495,6422.jpg,dog,0.0,1.000000e+02,100.0,16:44:43,2020-05-10
12496,3040.jpg,cat,100.0,0.000000e+00,100.0,16:44:43,2020-05-10
12497,10816.jpg,cat,100.0,0.000000e+00,100.0,16:44:43,2020-05-10
12498,2124.jpg,dog,0.0,1.000000e+02,100.0,16:44:43,2020-05-10


In [20]:
k = []
l = []

for i in os.listdir("test1/"):
    k.append(i)
    with open("test1/"+i, "rb") as img_file:
        my_string = base64.b64encode(img_file.read())
        l.append(my_string)
        
base_64 = pd.DataFrame(l)
image_name = pd.DataFrame(k)
base_64.columns = ["Image_string_base_64"]

In [21]:
df = pd.concat([df,base_64],axis =1)

In [22]:
df

,filename,category,Dog,Cat,Prediction,Time,Date,Image_string_base_64
0,11835.jpg,dog,0.0,1.000000e+02,100.0,16:44:42,2020-05-10,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAoHBwgHBgoI...
1,4737.jpg,dog,0.0,1.000000e+02,100.0,16:44:42,2020-05-10,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAoHBwgHBgoI...
2,3323.jpg,dog,0.0,1.000000e+02,100.0,16:44:42,2020-05-10,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAoHBwgHBgoI...
3,4758.jpg,cat,100.0,0.000000e+00,100.0,16:44:42,2020-05-10,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAoHBwgHBgoI...
4,5678.jpg,cat,100.0,0.000000e+00,100.0,16:44:42,2020-05-10,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAoHBwgHBgoI...
...,...,...,...,...,...,...,...,...
12495,6422.jpg,dog,0.0,1.000000e+02,100.0,16:44:43,2020-05-10,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAoHBwgHBgoI...
12496,3040.jpg,cat,100.0,0.000000e+00,100.0,16:44:43,2020-05-10,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAoHBwgHBgoI...
12497,10816.jpg,cat,100.0,0.000000e+00,100.0,16:44:43,2020-05-10,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAoHBwgHBgoI...
12498,2124.jpg,dog,0.0,1.000000e+02,100.0,16:44:43,2020-05-10,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAoHBwgHBgoI...


In [23]:
client = MongoClient('localhost', 27017)
db=client.Cat_dog_classifier
Model_Accuracy_95_prediction = db.Model_Accuracy_95_prediction

In [24]:
records_ = df.to_dict(orient = 'records')
result = db.Model_Accuracy_95_prediction.insert_many(records_ )